# Apendix

Here we show the code to calculate the transition matrix elements of a $d^4$ system like the one explored in chapter 5 for the ruthenate Ca2RuO4. First, we explore a small toy example to test the logic of the calculation and then we proceed to calculate the transition matrix elements of interest.

In [1]:
# Importing necessary extensions
import numpy as np
import itertools
import functools
import operator

from typing import List

In [2]:
def is_allowed(initial, final):
    jump = np.logical_xor(initial,final)  # creates the representation of where the spins changed
    # We must check whether only two changes were performed, meanign one jump.
    if np.unique(np.remainder(np.nonzero(jump),2)).size == 1:
        return 1
    return 0

## Exploration

Let us first explore the $d^4$ system. For this, we create the function to generate the list representation of states.

In [3]:
def generate_states(electrons: int, levels: int):
    """Generates the list representation of a given number of electrons and levels (degeneracy not considered)."""
    seed = [1 if position < electrons else 0 for position in range(levels)]
    generated_states = list(set(itertools.permutations(seed)))  # with set we filter out repeated states
    generated_states.sort(reverse=True)
    return generated_states

With this we can generate states of 3, 4, and 5 electrons in a 3 level system with degeneracy 2 meaning 6 levels in total.

In [4]:
states_d3 = generate_states(3,6)
states_d4 = generate_states(4,6)
states_d5 = generate_states(5,6)

We can consider first the $d^4$ states and take a look at them.

In [5]:
states_d4

[(1, 1, 1, 1, 0, 0),
 (1, 1, 1, 0, 1, 0),
 (1, 1, 1, 0, 0, 1),
 (1, 1, 0, 1, 1, 0),
 (1, 1, 0, 1, 0, 1),
 (1, 1, 0, 0, 1, 1),
 (1, 0, 1, 1, 1, 0),
 (1, 0, 1, 1, 0, 1),
 (1, 0, 1, 0, 1, 1),
 (1, 0, 0, 1, 1, 1),
 (0, 1, 1, 1, 1, 0),
 (0, 1, 1, 1, 0, 1),
 (0, 1, 1, 0, 1, 1),
 (0, 1, 0, 1, 1, 1),
 (0, 0, 1, 1, 1, 1)]

There is quite a list of generated states, but not all generated states are relevant for the problem. This means that we can reduce the amount of states beforehand. 

From all the $d^4$ states, we consider only those with a full $d_{xy}$ orbital and those which distribute the other two electrons in different orbitals as possible initial states since those are the possible ground states for the Ca2RuO4 system. In our representations, this means only the states that have a 1 in the first two entiresand no double occupancy in $zx$ or $yz$ orbitals.

In [6]:
possible_states_d4 = [list(state) for state in states_d4    # select from all the d4 states those that fulfill the following two contiditions
                      if state[0]==1 and state[1]==1        # 1) dxy orbital double occupancy
                      and state[2] is not state[3]          # 2) dzx/dyz orbital single occupancy
                     ]
possible_states_d4

[[1, 1, 1, 0, 1, 0],
 [1, 1, 1, 0, 0, 1],
 [1, 1, 0, 1, 1, 0],
 [1, 1, 0, 1, 0, 1]]

We obtain 4 different $d^4$ states that fullfill the conditions previously indicated.

Regarding the $d^3$ states, we want at least those which keep one electron in the $d_{xy}$ orbital since we know that the others states are not reachable with one movement as required by optical spectroscopy.

In [7]:
possible_states_d3 = [list(state) for state in states_d3 if state[0]==1 or state[1]==1]
possible_states_d3

[[1, 1, 1, 0, 0, 0],
 [1, 1, 0, 1, 0, 0],
 [1, 1, 0, 0, 1, 0],
 [1, 1, 0, 0, 0, 1],
 [1, 0, 1, 1, 0, 0],
 [1, 0, 1, 0, 1, 0],
 [1, 0, 1, 0, 0, 1],
 [1, 0, 0, 1, 1, 0],
 [1, 0, 0, 1, 0, 1],
 [1, 0, 0, 0, 1, 1],
 [0, 1, 1, 1, 0, 0],
 [0, 1, 1, 0, 1, 0],
 [0, 1, 1, 0, 0, 1],
 [0, 1, 0, 1, 1, 0],
 [0, 1, 0, 1, 0, 1],
 [0, 1, 0, 0, 1, 1]]

In the case of the $d^5$ states, since our ground state has a doule occupied $d_{xy}$ orbital then it has to stay occupied.

In [8]:
possible_states_d5 = [list(state) for state in states_d5 if state[0]==1 and state[1]==1]
possible_states_d5 

[[1, 1, 1, 1, 1, 0],
 [1, 1, 1, 1, 0, 1],
 [1, 1, 1, 0, 1, 1],
 [1, 1, 0, 1, 1, 1]]

We could generate all $d^3d^5$ combinations and check how many of them there are.

In [9]:
def combine_states(first: List, second: List):
    """Takes to lists of list representations of states and returns the list representation of a two-site state."""
    # Producing all the possible final states. This has to be read from bottom to top.
    final_states = [
                    functools.reduce(operator.add, combination)  # 3) the single site representations are combined into one single two-site representation
                            for combination                      # 2) we iterate over all the combinations produced
                            in itertools.product(first, second)  # 1) make the product of the given first and second states
                   ]
    
    final_states.sort(reverse=True)
    
    return final_states

print("The number of combined states is: ", len(combine_states(possible_states_d3,possible_states_d5)))

The number of combined states is:  64


We already saw in the previous section how we can check if a transition is allowed in our list codification of the states. Here we will make it a function slightly more flexible than what we had before.

In [10]:
def transition(initial: List, final: List[List[int]], debug = False):
    """This function takes the list representation of an initial state and a list of
    final d3 states. Then, it computes if the transition from the initial state to the 
    compounded d3d5 final state is possible.
    """
    
    final_states = combine_states(final, list(possible_states_d5))
    

        
    print("From initial state {}".format(initial))
    
    # We iterate over all final states and test whether the transition 
    # from the given initial state is allowed
    for state in final_states:
        allowed = is_allowed(initial, state)
        if allowed:
            print("       final state {} is allowed.".format(state))
        else:
            if debug:
                print("       final state {} not allowed".format(state))

In [11]:
transition(initial=[1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0], final=[[1,0,1,0,1,0],[0,1,0,1,0,1]])

From initial state [1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0]


With this, we can now explore the transitions between the different ground states and multiplets ($^4A_2$, $^2E_a$, $^2E_b$, $^2T_1$, and $^2T_2$ for the $d_3#).

Concerning the $d_4$ states, as explained in chapter X, there is the possibility to be in the $S_z=\pm1$ or $S_z=0$.

We will cover each one of them in the following.

## $d^3$ multiplet $^4A_2$

### $S_z=\pm1$

Starting with the pure $S_z=\pm1$ initial states, meaning $d_{\uparrow}^4d_{\uparrow}^4$ and $d_{\uparrow}^4d_{\downarrow}^4$, we have the following representations:

In [12]:
AFM_down = [1,1,0,1,0,1,1,1,1,0,1,0]
FM_down  = [1,1,0,1,0,1,1,1,0,1,0,1]

AFM_up = [1,1,1,0,1,0,1,1,0,1,0,1]
FM_up  = [1,1,1,0,1,0,1,1,1,0,1,0]

Then, the representation for a $|^4A_2,3/2>$ multiplet is given by

In [13]:
A2_32   = [[1,0,1,0,1,0], [0,1,0,1,0,1]]

and thus the transition between the initial states and the finals states given in the previous multiplet are

In [14]:
print("AFM states")
transition(AFM_down, A2_32)
transition(AFM_up, A2_32)
print("\nFM states")
transition(FM_down, A2_32)
transition(FM_up, A2_32)

AFM states
From initial state [1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0]
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]

FM states
From initial state [1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1]
       final state [0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1] is allowed.
       final state [0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1] is allowed.
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
       final state [1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0] is allowed.
       final state [1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1] is allowed.


We see that the AFM initial state has no transition matrix whereas the FM one is allowed. Comparing the initial and final states list representations and considering the $|^4A_2,3/2>$ prefactor, the matrix element is $t_{xy,xz}$.

Now we proceed with the $|^4A_2,1/2>$ multiple, which has the following representation:

In [15]:
A2_12 = [[1,0,0,1,0,1],[0,1,1,0,0,1],[0,1,0,1,1,0]]  # 4A2 Sz=1/2 d3 multiplet one spin up
print("AFM states")
transition(AFM_down, A2_12)
print("\nFM states")
transition(FM_down, A2_12)

AFM states
From initial state [1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0]
       final state [1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0] is allowed.
       final state [1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1] is allowed.

FM states
From initial state [1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1]


Here, only the AFM initial state is allowed. Once again, checking the prefactor for the multiplet we get a transition matrix element of $t_{xy,xz}/\sqrt{3}$.

When summing both contributions we obtain $\left(1+1\sqrt{3}\right)t_{xy,xz}$, as it could have been obtained easily by hand.

### $S_z=0$

The case of $S_z=0$ is handled similarly. We start with the $d_0^4d_\uparrow^4$ initial state. For this, we will split the $d_0^4$ in the two parts.

In [16]:
S0_1 = [1, 1, 1, 0, 0, 1]
S0_2 = [1, 1, 0, 1, 1, 0]

and then we add the $d^4_\uparrow$ representation to each one. Thus, for the $|^4A_2,3/2>$ $d^3$ multiplet we get

In [17]:
transition(S0_1 + [1, 1, 1, 0, 1, 0], A2_32)
transition(S0_2 + [1, 1, 1, 0, 1, 0], A2_32)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0]
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0]


that there is no allowed transition. doing the same for the $|^4A_2,1/2>$ $d_3$ multiplet

In [18]:
transition(S0_1 + [1, 1, 0, 1, 0, 1], A2_12)
transition(S0_2 + [1, 1, 0, 1, 0, 1], A2_12)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1]
       final state [0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1] is allowed.
       final state [0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1] is allowed.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1]
       final state [0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1] is allowed.
       final state [0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1] is allowed.


Here, both parts of the $S_z=0$ state contribute. Checking the prefactors for $S_z=0$ ($1/\sqrt{2}$) and $|^4A_2,1/2>$ ($1/\sqrt{3}$) we get a matrix element 

Continuing with the $d^4_0d^4_0$ the situation gets more complicated. In this case, we construct the four combinations for the initial state and calculate each one of them to later sum them up.

In [19]:
transition(S0_1 + S0_1, A2_32)
transition(S0_1 + S0_2, A2_32)
transition(S0_2 + S0_1, A2_32)
transition(S0_2 + S0_2, A2_32)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1]
From initial state [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0]
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1]
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]


No transitions from the $d^4_0d^4_0$ state to $|^4A_2,3/2>$.

And now, goint into the $|^4A_2,1/2>$ multiplet

In [20]:
transition(S0_1 + S0_1, A2_12)
transition(S0_1 + S0_2, A2_12)
transition(S0_2 + S0_1, A2_12)
transition(S0_2 + S0_2, A2_12)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1]
       final state [0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1] is allowed.
From initial state [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0]
       final state [0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0] is allowed.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1]
       final state [0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1] is allowed.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]
       final state [0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0] is allowed.


There are allowed transitions.

TODO: sum them

## $|^2E,a>$

First we encode the $|^2E,a>$ multiplet 

In [21]:
Ea = [[0,1,1,0,1,0],[1,0,0,1,1,0],[1,0,1,0,0,1]]
transition(AFM_down, Ea)
transition(AFM_up, Ea)
transition(FM_down, Ea)
transition(FM_up, Ea)

From initial state [1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0]
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]
       final state [0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1] is allowed.
       final state [0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1] is allowed.
From initial state [1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1]
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]


For Eb only transitions from the AFM state are possible. Collecting the prefactors we get that the transition matrix element in $-\sqrt{2/3}t_{xy,xz}$ as could be easily checked by hand.

In [22]:
Eb = [[1,0,1,0,0,1],[1,0,0,1,1,0]]
transition(AFM_down, Eb)
transition(AFM_up, Eb)
transition(FM_down, Eb)
transition(FM_up, Eb)

From initial state [1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0]
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]
From initial state [1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1]
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]


From the $S=\pm1$ initial states no transitions possible to Eb.

We follow with the situation when considering the $S=0$.

In [23]:
transition(S0_1 + S0_1, Ea)
transition(S0_1 + S0_2, Ea)
transition(S0_2 + S0_1, Ea)
transition(S0_2 + S0_2, Ea)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1]
       final state [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1] is allowed.
From initial state [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0]
       final state [1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1] is allowed.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1]
       final state [1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1] is allowed.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]
       final state [1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1] is allowed.


And Eb

In [24]:
transition(S0_1 + S0_1, Eb)
transition(S0_1 + S0_2, Eb)
transition(S0_2 + S0_1, Eb)
transition(S0_2 + S0_2, Eb)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1]
       final state [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1] is allowed.
From initial state [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0]
       final state [1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1] is allowed.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1]
       final state [1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1] is allowed.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]
       final state [1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1] is allowed.


## $|^2T1>$

First we encode the $|^2T1>$ multiplet 

In [25]:
T1 = [[1,0,0,0,1,1],[1,0,1,1,0,0]]
transition(AFM_down, T1)
transition(AFM_up, T1)
transition(FM_down, T1)
transition(FM_up, T1)

From initial state [1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0]
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]
From initial state [1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1]
From initial state [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]


In [26]:
transition(S0_1 + S0_1, T1)
transition(S0_1 + S0_2, T1)
transition(S0_2 + S0_1, T1)
transition(S0_2 + S0_2, T1)

From initial state [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1]
       final state [1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1] is allowed.
From initial state [1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0]
       final state [1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1] is allowed.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1]
       final state [1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1] is allowed.
From initial state [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0]
       final state [1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1] is allowed.
